In [70]:
# 02_log_prefix

In [71]:
# Force to reload extrernal modules every new cell execution
%reload_ext autoreload
%autoreload 2

In [72]:
### IMPORT ###
from pathlib import Path
import csv
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [73]:
### LOCAL IMPORT ###
from config import config_reader

In [74]:
### GLOBALS ###
yaml_config = config_reader.config_read_yaml("config.yml", "config")
log_dir = str(yaml_config["LOG_DIR"])
prefix_dir = str(yaml_config["LOG_PREFIX_DIR"])

log_in = "EVENT-LOG_ED_duration.csv" # <-- INPUT:  event log
dic_types = {"CaseID":object, "ACTIVITY":object, "TIMESTAMP":object, "RESOURCE":object, "ESI": int, "OUTCOME":object, "INPAT-HOSP-DEP":object, "CASE_DURATION_sec": float, "REMAINING_TIME_sec":float} # <-- INPUT: columns types
prefix_hour = 5 # <-- INPUT for prefix in hours

# FUNCTIONS

In [75]:
def filter_events_after_triage(event_log: pd.DataFrame, hours: int) -> pd.DataFrame:
    """
    Filters the event log to include only events that occurred within a specified number of hours
    after the first TRIAGE activity for each CaseID, including the TRIAGE activity itself.
    
    Parameters:
    event_log (pd.DataFrame): The original event log dataframe with columns including
                              'CaseID', 'ACTIVITY', 'TIMESTAMP', and others.
    hours (int): The number of hours within which events after the first TRIAGE activity are included.
    
    Returns:
    pd.DataFrame: A filtered dataframe containing the rows where the TRIAGE activity occurs
                  and subsequent rows where the event timestamp is within the specified number of
                  hours after the first TRIAGE activity for each CaseID.
    """
    # Convert TIMESTAMP column to datetime type
    event_log['TIMESTAMP'] = pd.to_datetime(event_log['TIMESTAMP'])

    # Define the function to filter rows for each CaseID
    def filter_within_hour_after_triage(df: pd.DataFrame) -> pd.DataFrame:
        # Find the minimum timestamp for TRIAGE activity
        triage_time = df.loc[df['ACTIVITY'] == 'TRIAGE', 'TIMESTAMP'].min()
        
        # If there is no TRIAGE activity, return an empty dataframe
        if pd.isna(triage_time):
            return pd.DataFrame()
        
        # Filter rows where the timestamp is within the specified number of hours after the TRIAGE time
        is_within_threshold = df['TIMESTAMP'] <= triage_time + pd.Timedelta(hours=hours)
        
        return df[is_within_threshold]

    # Apply the filter function to each group of CaseID
    filtered_event_log = event_log.groupby('CaseID').apply(lambda x: filter_within_hour_after_triage(x)).reset_index(drop=True)
    
    return filtered_event_log

In [76]:
def update_esi_counts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Updates the dataframe with ESI counts for each activity. ESI counts increase by one for each TRIAGE and
    decrease by one for each DISCHARGE, maintaining a cumulative count. The counts are then added back to the 
    original dataframe.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the event log with columns ['CaseID', 'ACTIVITY', 'TIMESTAMP', 'RESOURCE', 'ESI', 'OUTCOME', 'INPAT-HOSP-DEP', 'CASE_DURATION', 'REMAINING_TIME', 'COUNT-ESI-1', 'COUNT-ESI-2', 'COUNT-ESI-3', 'COUNT-ESI-4', 'COUNT-ESI-5']
    
    Returns:
    pd.DataFrame: Updated DataFrame with additional columns ['COUNT-ESI-1', 'COUNT-ESI-2', 'COUNT-ESI-3', 'COUNT-ESI-4', 'COUNT-ESI-5']
    """
    # Filter the dataframe to include only TRIAGE and DIMISSIONE activities
    df_filtered = df[df['ACTIVITY'].isin(['TRIAGE', 'DIMISSIONE'])]
    
    # Convert TIMESTAMP to datetime
    df_filtered['TIMESTAMP'] = pd.to_datetime(df_filtered['TIMESTAMP'])
    
    # Sort the dataframe by TIMESTAMP in ascending order (from less recent to most recent)
    df_filtered = df_filtered.sort_values(by='TIMESTAMP', ascending=True)
    
    # Initialize ESI counters
    esi_counters = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0
    }
    
    # Dictionaries to hold the updated counts for each row
    count_esi_1_dict = {}
    count_esi_2_dict = {}
    count_esi_3_dict = {}
    count_esi_4_dict = {}
    count_esi_5_dict = {}
    
    # Iterate over the filtered dataframe rows
    for index, row in df_filtered.iterrows():
        activity = row['ACTIVITY']
        esi = row['ESI']
        
        # Update the counter based on activity
        if activity == 'TRIAGE':
            esi_counters[esi] += 1
        elif activity == 'DIMISSIONE':
            esi_counters[esi] -= 1
        
        # Store the current counts in the dictionaries
        count_esi_1_dict[index] = esi_counters[1]
        count_esi_2_dict[index] = esi_counters[2]
        count_esi_3_dict[index] = esi_counters[3]
        count_esi_4_dict[index] = esi_counters[4]
        count_esi_5_dict[index] = esi_counters[5]
    
    # Create new columns in the original dataframe and set them to NaN
    df['COUNT-ESI-1'] = pd.NA
    df['COUNT-ESI-2'] = pd.NA
    df['COUNT-ESI-3'] = pd.NA
    df['COUNT-ESI-4'] = pd.NA
    df['COUNT-ESI-5'] = pd.NA
    
    # Update the new columns in the original dataframe with the counts from the dictionaries
    df.loc[df_filtered.index, 'COUNT-ESI-1'] = df_filtered.index.map(count_esi_1_dict)
    df.loc[df_filtered.index, 'COUNT-ESI-2'] = df_filtered.index.map(count_esi_2_dict)
    df.loc[df_filtered.index, 'COUNT-ESI-3'] = df_filtered.index.map(count_esi_3_dict)
    df.loc[df_filtered.index, 'COUNT-ESI-4'] = df_filtered.index.map(count_esi_4_dict)
    df.loc[df_filtered.index, 'COUNT-ESI-5'] = df_filtered.index.map(count_esi_5_dict)
    
    return df

In [77]:
def propagate_counts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Propagates the COUNT-ESI-x values from the TRIAGE activity to all other rows with the same CaseID.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the event log with columns ['CaseID', 'ACTIVITY', 'TIMESTAMP', 
                      'RESOURCE', 'ESI', 'OUTCOME', 'INPAT-HOSP-DEP', 'CASE_DURATION', 'REMAINING_TIME',
                      'COUNT-ESI-1', 'COUNT-ESI-2', 'COUNT-ESI-3', 'COUNT-ESI-4', 'COUNT-ESI-5']
    
    Returns:
    pd.DataFrame: DataFrame with COUNT-ESI-x values propagated.
    """
    # Iterate over each CaseID
    for case_id in df['CaseID'].unique():
        # Find the row with TRIAGE activity
        triage_row = df[(df['CaseID'] == case_id) & (df['ACTIVITY'] == 'TRIAGE')]
        if not triage_row.empty:
            triage_counts = triage_row[['COUNT-ESI-1', 'COUNT-ESI-2', 'COUNT-ESI-3', 'COUNT-ESI-4', 'COUNT-ESI-5']].iloc[0]
            # Propagate the counts to all rows with the same CaseID
            df.loc[df['CaseID'] == case_id, ['COUNT-ESI-1', 'COUNT-ESI-2', 'COUNT-ESI-3', 'COUNT-ESI-4', 'COUNT-ESI-5']] = triage_counts.values
    
    return df

# MAIN

In [78]:
### MAIN ###
print()
print("*** PROGRAM START ***")
print()

start_time = datetime.now().replace(microsecond=0)
print("Start process:", str(start_time))
print()

# print(yaml_config) # debug


*** PROGRAM START ***

Start process: 2024-08-01 11:33:12



In [79]:
print(">> Creating output directories")
directory_path = Path(prefix_dir) # <-- Specify the directory to be created
# Create the directory
directory_path.mkdir(parents=True, exist_ok=True)
print(f"Directory '{directory_path}' created successfully.")

>> Creating output directories
Directory 'data_prefix' created successfully.


In [80]:
print(">> Reading event log")
path_data = Path(log_dir) / log_in
print("File:", path_data)
df_log = pd.read_csv(path_data, sep=";", dtype=dic_types)
print("Event log shape:", df_log.shape)
print("Event log cases:", df_log["CaseID"].nunique())
print("Event log columns:", df_log.columns)

>> Reading event log
File: data_log/EVENT-LOG_ED_duration.csv
Event log shape: (20624, 9)
Event log cases: 3478
Event log columns: Index(['CaseID', 'ACTIVITY', 'TIMESTAMP', 'RESOURCE', 'ESI', 'OUTCOME',
       'INPAT-HOSP-DEP', 'CASE_DURATION_sec', 'REMAINING_TIME_sec'],
      dtype='object')


In [81]:
df_log

,CaseID,ACTIVITY,TIMESTAMP,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec
0,2022090001,TRIAGE,2022-09-01 00:03:55,NURS_0,3,A domicilio,-,41105.0,41105.0
1,2022090001,PRESA IN CARICO,2022-09-01 00:22:00,DOCT_0,3,A domicilio,-,41105.0,40020.0
2,2022090001,LABORATORIO,2022-09-01 00:28:21,-,3,A domicilio,-,41105.0,39639.0
3,2022090001,TC CRANIO,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0
4,2022090001,TAC,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0
...,...,...,...,...,...,...,...,...,...
20619,2022093478,DIMISSIONE,2022-10-01 00:21:00,-,3,A domicilio,-,2562.0,0.0
20620,2022093479,TRIAGE,2022-09-30 23:58:38,NURS_8,4,A domicilio,-,27322.0,27322.0
20621,2022093479,PRESA IN CARICO,2022-10-01 00:51:00,DOCT_1,4,A domicilio,-,27322.0,24180.0
20622,2022093479,LABORATORIO,2022-10-01 01:00:12,-,4,A domicilio,-,27322.0,23628.0


In [82]:
print(">> Filtering after n hours")
df_log_filtered = filter_events_after_triage(df_log, prefix_hour)
print("New event log shape:", df_log_filtered.shape)

>> Filtering after n hours
New event log shape: (17793, 9)


/var/folders/_n/gwd13q2j4zd4f80gls4mbcmw0000gn/T/ipykernel_7422/2393683643.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_event_log = event_log.groupby('CaseID').apply(lambda x: filter_within_hour_after_triage(x)).reset_index(drop=True)


In [83]:
# ESI
print(">> Extracting ESI values on the filtered event log")
esi_list = df_log_filtered['ESI'].unique().tolist()
esi_list.sort()
print("ESI unique values:", esi_list)

>> Extracting ESI values on the filtered event log
ESI unique values: [1, 2, 3, 4, 5]


In [84]:
df_log_filtered

,CaseID,ACTIVITY,TIMESTAMP,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec
0,2022090001,TRIAGE,2022-09-01 00:03:55,NURS_0,3,A domicilio,-,41105.0,41105.0
1,2022090001,PRESA IN CARICO,2022-09-01 00:22:00,DOCT_0,3,A domicilio,-,41105.0,40020.0
2,2022090001,LABORATORIO,2022-09-01 00:28:21,-,3,A domicilio,-,41105.0,39639.0
3,2022090001,TC CRANIO,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0
4,2022090001,TAC,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0
...,...,...,...,...,...,...,...,...,...
17788,2022093478,ELETTROCARDIOGRAMMA,2022-10-01 00:15:34,-,3,A domicilio,-,2562.0,326.0
17789,2022093478,DIMISSIONE,2022-10-01 00:21:00,-,3,A domicilio,-,2562.0,0.0
17790,2022093479,TRIAGE,2022-09-30 23:58:38,NURS_8,4,A domicilio,-,27322.0,27322.0
17791,2022093479,PRESA IN CARICO,2022-10-01 00:51:00,DOCT_1,4,A domicilio,-,27322.0,24180.0


In [85]:
print(">> Saving event log with new features")
log_out = f"{Path(log_in).stem}_prefix_{prefix_hour}h.csv"
path_out = Path(prefix_dir) / log_out
print("File:", path_out)
df_log_filtered.to_csv(path_out, sep=";", index=False)

>> Saving event log with new features
File: data_prefix/EVENT-LOG_ED_duration_prefix_5h.csv


In [86]:
# program end
end_time = datetime.now().replace(microsecond=0)
delta_time = end_time - start_time

print("End process:", end_time)
print("Time to finish:", delta_time)

print()
print("*** PROGRAM END ***")
print()

End process: 2024-08-01 11:33:13
Time to finish: 0:00:01

*** PROGRAM END ***

